In [3]:
import cv2
import numpy as np

# Load background subtractor
back_sub = cv2.createBackgroundSubtractorMOG2()

# Load YOLOv3
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load classes
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Load video
cap = cv2.VideoCapture("video.mp4")

# Define minimum confidence threshold
conf_threshold = 0.5

# Initialize variables
count = 0
car_positions = []

while True:
    # Read frames from the video
    ret, frame = cap.read()

    # Stop the loop if end of video
    if not ret:
        break

    # Apply background subtraction
    fg_mask = back_sub.apply(frame)

    # Apply morphological operations to reduce noise
    kernel = np.ones((5,5),np.uint8)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_OPEN, kernel)
    fg_mask = cv2.morphologyEx(fg_mask, cv2.MORPH_CLOSE, kernel)

    # Pass frame through network
    height, width, channels = frame.shape
    scale = 0.00392
    blob = cv2.dnn.blobFromImage(frame, scale, (416,416), (0,0,0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(net.getUnconnectedOutLayersNames())

    # Initialize lists for detected cars and their positions
    class_ids = []
    confidences = []
    car_boxes = []

    # Loop through all detections and filter for cars
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]

            # Filter for cars
            if classes[class_id] == "car" and confidence > conf_threshold:
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                # Check if detection is inside foreground mask
                if fg_mask[y:y+h,x:x+w].sum() > w * h * 0.4:
                    class_ids.append(class_id)
                    confidences.append(float(confidence))
                    car_boxes.append([x, y, w, h])

    # Apply non-max suppression to remove overlapping boxes
    indices = cv2.dnn.NMSBoxes(car_boxes, confidences, conf_threshold, 0.4)

    # Loop through filtered detections and count cars
    for i in indices.flatten():
        box = car_boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]

        matched = False
        for car_position in car_positions:
            if y > car_position[3] and y < car_position[3]+30:
                car_position[3] = y
                matched = True
                break

        if not matched:
            count += 1

            car_positions.append([x, y, x+w, y+h])

    # Display count on the video frame
    cv2.putText(frame, 'Vehicle count: {}'.format(count), (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Draw boxes around cars
    for i in indices.flatten():
        box = car_boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

     
    cv2.imshow('video2', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


In [4]:
import cv2
import numpy as np

# Load YOLOv3 network
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")

# Load names of all COCO objects
classes = []
with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

# Get indices of vehicle class
vehicle_class_ids = []
for i, cls in enumerate(classes):
    if cls in ['car', 'truck', 'bus', 'motorbike', 'bicycle']:
        vehicle_class_ids.append(i)

# Open video stream
cap = cv2.VideoCapture("video.mp4")

while True:
    # Read frame from video stream
    ret, frame = cap.read()
    if not ret:
        break

    # Get frame dimensions
    height, width, channels = frame.shape

    # Create blob from frame
    blob = cv2.dnn.blobFromImage(frame, 1/255.0, (416, 416), swapRB=True, crop=False)

    # Set input to network
    net.setInput(blob)

    # Get output from network
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)

    # Initialize lists for bounding boxes, confidences, and class IDs
    boxes = []
    confidences = []
    classIDs = []

    # Loop over each output layer
    for output in layerOutputs:
        # Loop over each detection in output layer
        for detection in output:
            # Get class ID and confidence
            scores = detection[5:]
            classID = np.argmax(scores)
            confidence = scores[classID]

            # Check if detection is a vehicle and has high enough confidence
            if classID in vehicle_class_ids and confidence > 0.5:
                # Increment vehicle count
                num_vehicles += 1
                cv2.putText(frame, str(num_vehicles), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,230), 2)
    # Print number of vehicles
    print(f"Number of vehicles: {num_vehicles}")
    cv2.imshow('video2', frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



# Release video stream
cap.release()


error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\dnn\src\darknet\darknet_importer.cpp:210: error: (-212:Parsing error) Failed to open NetParameter file: yolov3.cfg in function 'cv::dnn::dnn4_v20221220::readNetFromDarknet'


#####

### H ###

In [2]:
import cv2
import numpy as np
import time

cap = cv2.VideoCapture('WhatsApp Video 2023-04-17 at 11.10.50.mp4')


# initialize variables
prev_frame = None
prev_time = None



min_width_rect = 25 # min width rectangle
min_height_rect = 25 # min height rectangle

count_line_position = 325
# Initialize Subtractor
algo = cv2.bgsegm.createBackgroundSubtractorMOG()

# Load car detection classifier
car_classifier = cv2.CascadeClassifier('cars.xml')

def center_handle(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

detect = []
offset = 4# allowable offset
counter = 0

while True:
    ret, frame1 = cap.read()
    frame1 = cv2.resize(frame1, (540, 480))


    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)

    # store the first frame and time
    if prev_frame is None:
        prev_frame = blur
        prev_time = time.time()
        continue

    # calculate the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, blur)
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    # Find contours
    counterShape, h = cv2.findContours(dilatada, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Draw counting line
    cv2.line(frame1, (25, count_line_position), (1208, count_line_position), (255, 127, 0), 3)

    for (i,c) in enumerate(counterShape):
        x, y, w, h = cv2.boundingRect(c)

        # calculate the speed of the car
        
        validate_counter = (w >= min_width_rect) and (h >= min_height_rect)
        if not validate_counter:
            continue

        # Detect cars using the classifier
        car_rects = car_classifier.detectMultiScale(grey, 1.1, 3)

        for (x, y, w, h) in car_rects:

            if h > 30:
                curr_time = time.time()
                time_diff = curr_time - prev_time
                speed = (w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour
                prev_frame = blur
                prev_time = curr_time


            cv2.putText(frame1, f"Speed: {speed:.2f} mph", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Draw rectangle around detected object
            cv2.rectangle(frame1, (x, y), (x + w, y + h), (9, 255, 8), 2)
            center = center_handle(x, y, w, h)

            detect.append(center)
            cv2.circle(frame1, center, 4, (0, 8, 255), -1)

            for (x, y) in detect:
                # Check if the center of the detected object is within the counting area
                if y > count_line_position  and y < (count_line_position + offset):
                    counter += 1
                    detect.remove((x, y))
    
    cv2.putText(frame1, "VEHICLE COUNTER :"+str (counter), (45,47),cv2.FONT_HERSHEY_SIMPLEX, 1, (8,0,255),2)

    cv2.imshow('video2', frame1)
    
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\Dell\AppData\Local\Temp\ipykernel_14452\3084053577.py:81: RuntimeWarning: divide by zero encountered in divide
  speed = (w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour


In [38]:
counter

5

In [1]:
import cv2
import numpy as np
import time

# cap = cv2.VideoCapture('vehicle-counting.mp4')
cap = cv2.VideoCapture('D:\project colg\WhatsApp Video 2023-05-02 at 01.42.18.mp4')
# cap = cv2.VideoCapture('vehicle-counting.mp4')

# initialize variables
prev_frame = None
prev_time = None
min_width_rect = 10 # min width rectangle
min_height_rect = 10 # min height rectangle
count_line_position = 325
# Initialize Subtractor
algo = cv2.bgsegm.createBackgroundSubtractorMOG()
car_count = 0
# Load car detection classifier
car_classifier = cv2.CascadeClassifier('cars.xml')

def center_handle(x, y, w, h):
    x1 = int(w / 2)
    y1 = int(h / 2)
    cx = x + x1
    cy = y + y1
    return cx, cy

detect = []
offset = 4# allowable offset
counter = 0

while True:
    ret, frame1 = cap.read()
    if not ret:
        break
    frame1 = cv2.resize(frame1, (640, 480))
    frame1_copy = frame1.copy()

    grey = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(grey, (3, 3), 5)

    # store the first frame and time
    if prev_frame is None:
        prev_frame = blur
        prev_time = time.time()
        continue

    # calculate the absolute difference between the current frame and the previous frame
    diff = cv2.absdiff(prev_frame, blur)
    # applying on each frame
    img_sub = algo.apply(blur)
    dilat = cv2.dilate(img_sub, np.ones((5, 5)))
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    dilatada = cv2.morphologyEx(dilat, cv2.MORPH_CLOSE, kernel)
    dilatada = cv2.morphologyEx(dilatada, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, hierarchy = cv2.findContours(dilatada, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw counting line
    # cv2.line(frame1_copy, (25, count_line_position), (1208, count_line_position), (255, 127, 0), 3)

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)

        # validate size of detected object
        validate_counter = (w >= min_width_rect) and (h >= min_height_rect)
        if not validate_counter:
            continue

        # calculate the speed of the car
        car_roi = grey[y-10:y+h+10, x-10:x+w+10]
        car_rects = car_classifier.detectMultiScale(grey, 1.1, 3)

        for (car_x, car_y, car_w, car_h) in car_rects:
            if car_h > 20:
                curr_time = time.time()
                time_diff = curr_time - prev_time
                speed = (car_w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour
                prev_frame = blur
                prev_time = curr_time

            cv2.putText(frame1_copy, f"Speed: {speed:.2f} mph", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            # Draw rectangle around detected object
            cv2.rectangle(frame1_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)

        # check if the center of the bounding box crossed the counting line
        center_y = int(y + h / 2)
        if center_y >= count_line_position and center_y <= count_line_position + 5:
            car_count += 1

    # Display car count
    cv2.putText(frame1_copy, f"Car count: {car_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    # Display the resulting frame
    cv2.imshow('Traffic Analysis', frame1_copy)
    if cv2.waitKey(1) == ord('q'):
        break
#Release the capture and close the window
cap.release()
cv2.destroyAllWindows()


C:\Users\Dell\AppData\Local\Temp\ipykernel_14452\1847956922.py:79: RuntimeWarning: divide by zero encountered in divide
  speed = (car_w / time_diff) * 0.0268224  # conversion factor from pixels per second to miles per hour


# Number Plate

In [1]:
import cv2
import easyocr

# Load video file
cap = cv2.VideoCapture('WhatsApp Video 2023-05-02 at 01.42.18.mp4')
reader = easyocr.Reader(['en'])

# Set video frame size
cap.set(3, 640)
cap.set(4, 480)

# Load pre-trained Haar Cascade classifier
plate_cascade = cv2.CascadeClassifier('NUM_PLATE.xml')
wanted = ['R-183-JF']

while True:
    # Read frame from video
    ret, frame = cap.read()
    frame = cv2.resize(frame, (640, 480))
    frame1=frame
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect number plates using Haar Cascade classifier
    plates = plate_cascade.detectMultiScale(gray, 1.3, 5)

    # Loop through detected number plates
    for (x, y, w, h) in plates:
        # Crop number plate from frame
        plate_img = gray[y:y + h, x:x + w]

        # Apply thresholding to enhance characters in number plate
        thresh = cv2.threshold(plate_img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

        #  read the number plate text
        plate_text = reader.readtext(thresh, paragraph=False)

        # Draw bounding box around number plate and display number plate text
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        try:

            cv2.putText(frame, str(plate_text[0][1]), (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 4)
            if plate_text[0][1] in wanted:
                cv2.putText(frame, "SUSPICIOUS NUMBER PLATE DETECTED", (20, 25), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 4)
            
        except:
            # cv2.putText(frame, "Can't read", (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            pass
    # Display video frame
    cv2.imshow('frame', frame)
    

    # Exit if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release video capture and destroy windows
cap.release()
cv2.destroyAllWindows()q

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.
